In [1]:
import os

import re
import time
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta

from utils.funcs import main_work

from dotenv import load_dotenv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver import ActionChains

load_dotenv()


# ElementClickInterceptedException - неможна клацнуть по елементу
# NoSuchElementException - такого елементу неіснує

True

In [2]:
# funcs

def login(driver):
    email = os.getenv("EMAIL")
    password = os.getenv("PASS")
    
    
    login_button = find_elems(driver, {
                                        "by_attr": By.ID, 
                                        "attr_data": "user-menu",
                                        "attr_target": "div"
                                        }
                                )[0]
    login_button.click()
    
    
    block_login_by = find_elems(driver, {
                                        "by_attr": By.CLASS_NAME, 
                                        "attr_data": "lsidDialog__section",
                                        }
                               )
    login_by_email = find_elems(block_login_by[0], {
                                        "by_attr": By.TAG_NAME, 
                                        "attr_data": "button",
                                        }
                               )[-1]
    login_by_email.click()
    
    block_inputs = find_elems(driver, {
                                        "by_attr": By.CLASS_NAME, 
                                        "attr_data": "lsidDialog__section",
                                        }
                               )
    input_fields = find_elems(block_inputs[0], {
                                        "by_attr": By.TAG_NAME, 
                                        "attr_data": "input",
                                        }
                             )
    time.sleep(1)
    input_fields[0].send_keys(email)
    input_fields[1].send_keys(password)
    input_fields[1].send_keys(Keys.RETURN)  
      
    time.sleep(5)  # для закриття вспливаючого вікна
    return driver


def get_driver():
    path_to_driver = "./chromedriver"
    chrome_service = Service(path_to_driver)
    options = Options()
#     options.add_argument("headless")
    return webdriver.Chrome(
        service=chrome_service,
        options=options
    )


def close_cookies(driver):
    """
    закриття вікна з підтвердження куків
    """
    
    elems = find_elems(driver, {
                                "by_attr": By.ID, 
                                "attr_data": "onetrust-accept-btn-handler",
                                }
                      )
    if not elems:
        return driver
    elems[0].click()
    return driver
    
    
def find_elems(target, args):
    args_attr_data = args["attr_data"]
    if args["by_attr"] == By.XPATH and args["by_text"] is False:
        attr_data =  f"//{'attr_target'}[starts-with(@class, '{args_attr_data}')]" 
    elif args["by_attr"] == By.XPATH and args["by_text"] is True:
        attr_data =  f"//{'attr_target'}[text()='{args_attr_data}']"
    else:
        attr_data = args["attr_data"]
    
    count = 0
    while count <= 5:
        try:
            elements = target.find_elements(args["by_attr"], attr_data)
            if len(elements) == 0:
                count += 1
                continue
            break
        except NoSuchElementException:
            time.sleep(1)
            count += 1
    
    return elements


def get_my_champs(driver):
    """
    створення списку кортежів, назв чемпів + частин посилання, на всі чемпи в моїх лігах
    """
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    champs = []
    for champ in soup.find_all("div", class_="leftMenu__item"):
        href = champ.find('a', class_="leftMenu__href")["href"]
        champs.append((champ["title"], href))
    return champs


def get_champ_dict_from_file():
    champ_dict = {}
    with open("champ_list.txt", "r") as f:
        for line in f:
            name, href = line.split(",")
            champ_dict[name.strip()] = href.strip()
    return champ_dict


def get_future_matches(driver, days=0):
    # переключення дня 
    # допрацювати
    for _ in range(days):
        pass
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    future_matches_block = soup.find_all("div", class_=re.compile("event__header top"))
    
    future_matches_dict = {}
    for champ_block in future_matches_block: 
        champ_country_name = champ_block.find("span", class_="event__title--type").text # назва чемпа країни
        champ_name = champ_block.find("span", class_="event__title--name").text # назва чемпа
        full_champ_name = f"{champ_country_name}: {champ_name}"
        
        future_matches_dict[full_champ_name] = []
        
        next_div = champ_block.find_next_sibling("div")
        while "id" in next_div.attrs.keys():
            future_matches_dict[full_champ_name].append((
                next_div.find("div", class_=re.compile("event__participant event__participant--home")).text,
                next_div.find("div", class_=re.compile("event__participant event__participant--away")).text
            ))
            
            next_div = next_div.find_next_sibling("div")
        
    return future_matches_dict
#     return soup 
    

def handle_one_champ(driver, champ):
    url_future_matches = f"https://www.soccerstand.com{champ}fixtures/"
    url_past_matches = f"https://www.soccerstand.com{champ}results/"
    driver.get(url_past_matches)  
    time.sleep(3)
    return driver.page_source
    # допрацювати   
    
    
def get_df_for_champ(data):
    """
    створення датафрейму даних результатів одного чемпа одного 

    """
    soup = BeautifulSoup(data, 'html.parser')
    matches_block = soup.find_all("div", title="Подробности матча!")
    full_champ_data = []
    for match_block in matches_block:
        try:
            a = match_block.find("div", class_="event__time").text

            d, m, _ = [ x.strip() for x in a.split(".")]
            y = date.today().year if int(m) <= date.today().month else date.today().year - 1

            h_name = match_block.find("div", class_=re.compile("event__participant event__participant--home")).text
            h_full_score = match_block.find("div", class_=re.compile("event__score event__score--home")).text
            h_first_score = match_block.find(
                                            "div", 
                                            class_=re.compile("event__part event__part--home event__part--1")
                                        ).text.strip("()")

            a_name = match_block.find("div", class_=re.compile("event__participant event__participant--away")).text
            a_full_score = match_block.find("div", class_=re.compile("event__score event__score--away")).text
            a_first_score = match_block.find(
                                            "div", 
                                            class_=re.compile("event__part event__part--away event__part--1")
                                        ).text.strip("()")
            full_date = f"{d}/{m}/{y}"
 
            full_champ_data.append((full_date, 
                                    h_name, a_name,
                                    h_full_score, a_full_score ,
                                    h_first_score, a_first_score
                                ))
        except AttributeError:
            continue

    return pd.DataFrame(
        data=full_champ_data,
        columns=["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HTHG", "HTAG"]
    )

In [13]:
try:
    url = "https://www.soccerstand.com/ru/"
    driver = get_driver()
    driver.get(url)
    driver = close_cookies(driver)
    
    driver = login(driver)  

    # отримання списку(словаря) моїх чемпіонатів
#     champs = get_my_champs(driver) # отримання списку(словаря) з сайта
    champ_dict = get_champ_dict_from_file()
    
    future_matches_dict = get_future_matches(driver, days=0)

#     champ_list = [x for x in future_matches_dict.keys() if x in champ_dict.keys()]
#     for champ in champ_list:

    for champ in future_matches_dict.keys():
#     for champ in ["АВСТРАЛИЯ: НПЛ АСТ"]:
        try:
            data = handle_one_champ(driver, champ_dict[champ])
        except KeyError:
            try:
                new_champ = list(filter(
                                        lambda x: champ.startswith(x),
                                        champ_dict.keys()  
                            ))[0]
                data = handle_one_champ(driver, champ_dict[new_champ])
            except:
                print(f"WATAFAK - {champ}")
                continue
        print(champ, future_matches_dict[champ])
#         print(champ, [('Канберра Олимпик', 'Гангалин')])
        
        print()
        try:
            df = get_df_for_champ(data)
            
            main_work(df, future_matches_dict[champ])
    #         main_work(df, [('Канберра Олимпик', 'Гангалин')])
        except Exception as e:
            print(e)
            continue
        time.sleep(1)
#         raise
    
finally:
    driver.quit()


АВСТРАЛИЯ: НПЛ АСТ [('Канберра Олимпик', 'Гангалин')]

останні резульати команд - Канберра Олимпик - Гангалин
1 4 -4 -2

result - no
АВСТРАЛИЯ: НПЛ Новый Южный Уэльс [('Блэктаун Сити', 'Централ Кост U21'), ('Рокдейл Илинден', 'Сидней Олимпик')]

останні резульати команд - Блэктаун Сити - Централ Кост U21
-3 -1 0 0

result - no
останні резульати команд - Рокдейл Илинден - Сидней Олимпик
1 0 1 -1

result - no
АВСТРАЛИЯ: НПЛ Квинсленд [('Брисбен U21', 'Голд Коаст Юнайтед'), ('Рочдейл Роверс', 'Мортон-Бэй'), ('Голд-Кост Найтс', 'Истерн Сабербс')]

too many values to unpack (expected 3)
АВСТРАЛИЯ: НПЛ Южная Австралия [('Аделаида U21', 'Стюрт Лайонс')]

останні резульати команд - Аделаида U21 - Стюрт Лайонс
-1 0 -1 0

result - no
АВСТРАЛИЯ: НПЛ Виктория [('Сент Олбанс', 'Хьюм Сити'), ('Саут Мельбурн', 'Данденонг')]

останні резульати команд - Сент Олбанс - Хьюм Сити
-1 -3 -1 -1

result - no
останні резульати команд - Саут Мельбурн - Данденонг
1 1 0 -1

result - no
АВСТРИЯ: Бундеслига - Confe

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.74)
Stacktrace:
#0 0x5608f31cc303 <unknown>
#1 0x5608f2fa0d37 <unknown>
#2 0x5608f2f7b975 <unknown>
#3 0x5608f30025df <unknown>
#4 0x5608f3015253 <unknown>
#5 0x5608f2ffd753 <unknown>
#6 0x5608f2fd0a14 <unknown>
#7 0x5608f2fd1b7e <unknown>
#8 0x5608f321b32e <unknown>
#9 0x5608f321ec0e <unknown>
#10 0x5608f3201610 <unknown>
#11 0x5608f321fc23 <unknown>
#12 0x5608f31f3545 <unknown>
#13 0x5608f32406a8 <unknown>
#14 0x5608f3240836 <unknown>
#15 0x5608f325bd13 <unknown>
#16 0x7faf37a62b43 <unknown>


In [5]:
future_matches_dict

NameError: name 'future_matches_dict' is not defined

In [53]:
champ_dict

{'АВСТРАЛИЯ: Лига А': '/ru/football/australia/a-league/',
 'АВСТРАЛИЯ: НПЛ АСТ': '/ru/football/australia/npl-act/',
 'АВСТРАЛИЯ: НПЛ Северный Новый Южный Уэльс': '/ru/football/australia/npl-northern-nsw/',
 'АВСТРАЛИЯ: НПЛ Новый Южный Уэльс': '/ru/football/australia/npl-nsw/',
 'АВСТРАЛИЯ: НПЛ Квинсленд': '/ru/football/australia/npl-queensland/',
 'АВСТРАЛИЯ: НПЛ Южная Австралия': '/ru/football/australia/npl-south-australian/',
 'АВСТРАЛИЯ: НПЛ Тасмания': '/ru/football/australia/npl-tasmania/',
 'АВСТРАЛИЯ: НПЛ Виктория': '/ru/football/australia/npl-victoria/',
 'АВСТРАЛИЯ: НПЛ Западная Австралия': '/ru/football/australia/npl-western-australia/',
 'АВСТРИЯ: Бундеслига': '/ru/football/austria/bundesliga/',
 'АВСТРИЯ: Вторая лига': '/ru/football/austria/2-liga/',
 'АЗЕРБАЙДЖАН: Премьер-лига': '/ru/football/azerbaijan/premier-league/',
 'АЛБАНИЯ: Суперлига': '/ru/football/albania/super-league/',
 'АЛЖИР: Первый дивизион': '/ru/football/algeria/division-1/',
 'АЛЖИР: Лига 2': '/ru/football

In [12]:
get_df_for_champ(data)


ValueError: too many values to unpack (expected 3)

In [8]:
a.to_csv('./old_data/some_test.csv', index=False)

In [6]:
main_work(df, [('Брисбен U21', 'Голд Коаст Юнайтед'), ('Рочдейл Роверс', 'Мортон-Бэй'), ('Голд-Кост Найтс', 'Истерн Сабербс')])


AttributeError: 'datetime.date' object has no attribute 'split'